# NER
不让llm去确认位置，我自己用算法后期加上。

In [1]:
import json

with open('/home/hbchen/Projects/MedKGC/resource/radgraph/test.json', 'r') as f:
    data = json.load(f)
json_result = None
text = None

index = 0
for key, value in data.items():
    json_result = value['labeler_1']['entities']
    text = value['text']

    break
    index += 1
    
    if index == 4:
        break

print(text)

def tuple_from_radgraph(json_result):
    result = {}
    for key, value in json_result.items():
        result[value['tokens']] = value['label']
    return result

print(json_result)
result = tuple_from_radgraph(json_result)
result

FINAL REPORT INDICATION : ___ F with cough / / Cough TECHNIQUE : PA and lateral views of the chest . COMPARISON : None . FINDINGS : The lungs are clear without focal consolidation , , or edema . The cardiomediastinal silhouette is within normal limits . No acute osseous abnormalities . IMPRESSION : No acute cardiopulmonary process .
{'1': {'tokens': 'lungs', 'label': 'ANAT-DP', 'start_ix': 28, 'end_ix': 28, 'relations': []}, '2': {'tokens': 'clear', 'label': 'OBS-DP', 'start_ix': 30, 'end_ix': 30, 'relations': [['located_at', '1']]}, '3': {'tokens': 'focal', 'label': 'OBS-DA', 'start_ix': 32, 'end_ix': 32, 'relations': [['modify', '4']]}, '4': {'tokens': 'consolidation', 'label': 'OBS-DA', 'start_ix': 33, 'end_ix': 33, 'relations': [['located_at', '1']]}, '5': {'tokens': 'edema', 'label': 'OBS-DA', 'start_ix': 37, 'end_ix': 37, 'relations': [['located_at', '1']]}, '6': {'tokens': 'cardiomediastinal', 'label': 'ANAT-DP', 'start_ix': 40, 'end_ix': 40, 'relations': []}, '7': {'tokens': 's

{'lungs': 'ANAT-DP',
 'clear': 'OBS-DP',
 'focal': 'OBS-DA',
 'consolidation': 'OBS-DA',
 'edema': 'OBS-DA',
 'cardiomediastinal': 'ANAT-DP',
 'silhouette': 'ANAT-DP',
 'within': 'OBS-DP',
 'normal': 'OBS-DP',
 'limits': 'OBS-DP',
 'acute': 'OBS-DA',
 'osseous': 'ANAT-DP',
 'abnormalities': 'OBS-DA',
 'cardiopulmonary': 'ANAT-DP',
 'process': 'OBS-DA'}

# Prompt

In [25]:
import json
import requests

# Create a session object
session = requests.Session()

def extract_entities(text):
  url = 'http://ollama.corinth.informatik.rwth-aachen.de/api/chat'

  data = {
    "model": "llama3.1:8b",
    "messages": [
      {
        "role": "system",
        "content": '''You are a computer scientist and radiology expert specializing in named entity recognition (NER) and relation extraction. Your task is to extract entities from the provided radiology report text and classify them into one of the following labels: ANAT-DP, OBS-DP, OBS-U, OBS-DA.

Here are some examples of how entities should be extracted and classified:

Example 1:
Input text:
"FINAL REPORT INDICATION: ___ F with cough / / Cough TECHNIQUE: PA and lateral views of the chest. COMPARISON: None. FINDINGS: The lungs are clear without focal consolidation, or edema. The cardiomediastinal silhouette is within normal limits. No acute osseous abnormalities. IMPRESSION: No acute cardiopulmonary process."

Extracted entities:
{
 'lungs': 'ANAT-DP',
 'clear': 'OBS-DP',
 'focal': 'OBS-DA',
 'consolidation': 'OBS-DA',
 'edema': 'OBS-DA',
 'cardiomediastinal': 'ANAT-DP',
 'silhouette': 'ANAT-DP',
 'within': 'OBS-DP',
 'normal': 'OBS-DP',
 'limits': 'OBS-DP',
 'acute': 'OBS-DA',
 'osseous': 'ANAT-DP',
 'abnormalities': 'OBS-DA',
 'cardiopulmonary': 'ANAT-DP',
 'process': 'OBS-DA'
}

Example 2:
Input text:
"FINAL REPORT HISTORY: Chest tube leak, to assess for pneumothorax. FINDINGS: In comparison with study of ___, the endotracheal tube and Swan-Ganz catheter have been removed. The left chest tube remains in place and there is no evidence of pneumothorax. Mild atelectatic changes are seen at the left base."

Extracted entities:
{
 'endotracheal': 'OBS-DA',
 'tube': 'OBS-DP',
 'Swan - Ganz': 'OBS-DA',
 'catheter': 'OBS-DA',
 'left': 'ANAT-DP',
 'chest': 'ANAT-DP',
 'in place': 'OBS-DP',
 'pneumothorax': 'OBS-DA',
 'Mild': 'OBS-DP',
 'atelectatic': 'OBS-DP',
 'changes': 'OBS-DP',
 'base': 'ANAT-DP'
}

Now, apply this classification to the provided text and return the result in JSON format as shown below:

{
    "1": {"tokens": "Lungs", "label": "ANAT-DP"},
    "2": {"tokens": "clear", "label": "OBS-DP"},
    ...
}

        
        please only output the json, do not include any other information in the output.'''
      },
      {
        "role": "user",
        "content": text
      }
    ],
    "stream": False
  }

  response = session.post(url, json=data)
  entities = response.json()['message']['content']

  return json.loads(entities)

# Example usage
with open('/home/hbchen/Projects/MedKGC/resource/radgraph/test.json', 'r') as f:
    data = json.load(f)

json_result = None
text = None

for key, value in data.items():
    json_result = value['labeler_1']['entities']
    text = value['text']
    break
print(text)

entities = extract_entities(text)

entities

2024-10-21 10:52:21 urllib3.connectionpool DEBUG: Starting new HTTP connection (1): ollama.corinth.informatik.rwth-aachen.de:80


FINAL REPORT INDICATION : ___ F with cough / / Cough TECHNIQUE : PA and lateral views of the chest . COMPARISON : None . FINDINGS : The lungs are clear without focal consolidation , , or edema . The cardiomediastinal silhouette is within normal limits . No acute osseous abnormalities . IMPRESSION : No acute cardiopulmonary process .


2024-10-21 10:52:25 urllib3.connectionpool DEBUG: http://ollama.corinth.informatik.rwth-aachen.de:80 "POST /api/chat HTTP/1.1" 200 1263


{'1': {'tokens': 'lungs', 'label': 'ANAT-DP'},
 '2': {'tokens': 'clear', 'label': 'OBS-DP'},
 '3': {'tokens': 'focal', 'label': 'OBS-DA'},
 '4': {'tokens': 'consolidation', 'label': 'OBS-DA'},
 '5': {'tokens': 'edema', 'label': 'OBS-DA'},
 '6': {'tokens': 'cardiomediastinal', 'label': 'ANAT-DP'},
 '7': {'tokens': 'silhouette', 'label': 'ANAT-DP'},
 '8': {'tokens': 'within', 'label': 'OBS-DP'},
 '9': {'tokens': 'normal', 'label': 'OBS-DP'},
 '10': {'tokens': 'limits', 'label': 'OBS-DA'},
 '11': {'tokens': 'acute', 'label': 'OBS-DA'},
 '12': {'tokens': 'osseous', 'label': 'ANAT-DP'},
 '13': {'tokens': 'abnormalities', 'label': 'OBS-DA'},
 '14': {'tokens': 'cardiopulmonary', 'label': 'ANAT-DP'},
 '15': {'tokens': 'process', 'label': 'OBS-DA'}}

# Run

In [26]:
from ner_eval import compute_metrics
from ner_eval import Entity
import pprint   

## Auxilary functions

In [27]:
import json
from collections import namedtuple

Entity = namedtuple("Entity", "e_type start_offset end_offset")


def transform_radgraph_to_entity(json_result):
    entities = []

    for key, value in json_result.items():
        e_type = value['label']
        start_offset = value['start_ix']
        end_offset = value['end_ix']
        entities.append(Entity(e_type=e_type, start_offset=start_offset, end_offset=end_offset))

    return entities


def transform_llm_response_to_entity(json_result, text):
    entities = []
    words = text.split()
    word_index = 0
    
    for key, value in json_result.items():
        token = value['tokens']
        e_type = value['label']

        # Find the start and end word offsets of the token in the text
        while word_index < len(words):
            if words[word_index] == token:
                start_offset = word_index
                end_offset = word_index
                entities.append(Entity(e_type=e_type, start_offset=start_offset, end_offset=end_offset))
                word_index += 1
                break
            word_index += 1
    
    return entities


# Example usage
with open('/home/hbchen/Projects/MedKGC/resource/radgraph/test.json', 'r') as f:
    data = json.load(f)

json_result = None
text = None

index = 0
for key, value in data.items():
    json_result = value['labeler_1']['entities']
    text = value['text']
    
    # break
    index += 1
    if index == 4:
        break
text

'FINAL REPORT HISTORY : Chest tube leak , to assess for pneumothorax . FINDINGS : In comparison with study of ___ , the endotracheal tube and Swan - Ganz catheter have been removed . The left chest tube remains in place and there is no evidence of pneumothorax . Mild atelectatic changes are seen at the left base .'

In [28]:
y_true = transform_radgraph_to_entity(json_result)
y_true

[Entity(e_type='OBS-DA', start_offset=23, end_offset=23),
 Entity(e_type='OBS-DA', start_offset=24, end_offset=24),
 Entity(e_type='OBS-DA', start_offset=26, end_offset=28),
 Entity(e_type='OBS-DA', start_offset=29, end_offset=29),
 Entity(e_type='ANAT-DP', start_offset=35, end_offset=35),
 Entity(e_type='ANAT-DP', start_offset=36, end_offset=36),
 Entity(e_type='OBS-DP', start_offset=37, end_offset=37),
 Entity(e_type='OBS-DP', start_offset=39, end_offset=40),
 Entity(e_type='OBS-DA', start_offset=47, end_offset=47),
 Entity(e_type='OBS-DP', start_offset=49, end_offset=49),
 Entity(e_type='OBS-DP', start_offset=50, end_offset=50),
 Entity(e_type='OBS-DP', start_offset=51, end_offset=51),
 Entity(e_type='ANAT-DP', start_offset=56, end_offset=56),
 Entity(e_type='ANAT-DP', start_offset=57, end_offset=57)]

In [29]:
# print(json_result
print("transform_llm_response_to_entity")
json_result = extract_entities(text)
print(json_result)

pred = transform_llm_response_to_entity(json_result, text)
pred

transform_llm_response_to_entity


2024-10-21 10:52:41 urllib3.connectionpool DEBUG: http://ollama.corinth.informatik.rwth-aachen.de:80 "POST /api/chat HTTP/1.1" 200 1240


{'1': {'tokens': 'Chest', 'label': 'ANAT-DP'}, '2': {'tokens': 'tube', 'label': 'OBS-DA'}, '3': {'tokens': 'leak', 'label': 'OBS-DA'}, '4': {'tokens': 'pneumothorax', 'label': 'OBS-DA'}, '5': {'tokens': 'endotracheal', 'label': 'OBS-DA'}, '6': {'tokens': 'tube', 'label': 'OBS-DP'}, '7': {'tokens': 'Swan - Ganz', 'label': 'OBS-DA'}, '8': {'tokens': 'catheter', 'label': 'OBS-DA'}, '9': {'tokens': 'left', 'label': 'ANAT-DP'}, '10': {'tokens': 'chest', 'label': 'ANAT-DP'}, '11': {'tokens': 'in place', 'label': 'OBS-DP'}, '12': {'tokens': 'mild', 'label': 'OBS-DP'}, '13': {'tokens': 'atelectatic', 'label': 'OBS-DA'}, '14': {'tokens': 'changes', 'label': 'OBS-DA'}, '15': {'tokens': 'base', 'label': 'ANAT-DP'}}


[Entity(e_type='ANAT-DP', start_offset=4, end_offset=4),
 Entity(e_type='OBS-DA', start_offset=5, end_offset=5),
 Entity(e_type='OBS-DA', start_offset=6, end_offset=6),
 Entity(e_type='OBS-DA', start_offset=11, end_offset=11),
 Entity(e_type='OBS-DA', start_offset=23, end_offset=23),
 Entity(e_type='OBS-DP', start_offset=24, end_offset=24)]

In [30]:
tag = ['OBS-DP', 'ANAT-DP', 'OBS-U', 'OBS-DA']
result, evaluation_agg_entities_type = compute_metrics(y_true, pred, tag)

result['strict']

{'correct': 1,
 'incorrect': 1,
 'partial': 0,
 'missed': 12,
 'spurious': 4,
 'precision': 0,
 'recall': 0,
 'actual': 6,
 'possible': 14}

# Evaluation
之前的evaluation是用的token-level的F1，原因是bert的输出是token-level的。

现在改成了word-level的F1。

## Macro F1
定义：Macro F1 是对每个类别单独计算 F1 分数后，再进行算术平均。它给予每个类别相同的重要性，因此在类别不平衡时表现更佳。

我们并不知道任何分类的信息，无法计算 Macro F1。

## Micro F1

In [ ]:
import json
import pprint

def compare(result_entity, target):
    tokens = result_entity["tokens"]
    label = result_entity["label"]

    target_tokens = target["tokens"]
    target_label = target["label"]

    # For entity recognition, a predicted entity is considered correct if the predicted span boundaries and predicted entity type are both correct
    if tokens == target_tokens and label == target_label:
        return True

    return result_entity == target

import nereval
from nereval import Entity

def prepare_target_from_radgraph(data):
    entities_list = []
    
    for file_key, file_value in data.items():
        if 'labeler_1' in file_value:
            labeler_1 = file_value['labeler_1']
            if 'entities' in labeler_1:
                entities = labeler_1['entities']
                for entity_id, entity_info in entities.items():
                    entity = Entity(
                        entity_info['tokens'],
                        entity_info['label'],
                        entity_info['start_ix']
                    )
                    entities_list.append(entity)
        elif 'entities' in file_value:
            entities = file_value['entities']
            for entity_id, entity_info in entities.items():
                entity = Entity(
                    entity_info['tokens'],
                    entity_info['label'],
                    entity_info['start_ix']
                )
                entities_list.append(entity)
    
    return entities_list

# Example usage
with open('/home/hbchen/Projects/MedKGC/resource/radgraph/test.json', 'r') as f:
    data = json.load(f)

y_true = prepare_target_from_radgraph(data)
pprint.pprint(entities)


def prepare_result_from_text(result, text):
    entities_list = []
    
    for entity_id, entity_info in result.items():
        entity = Entity(
            entity_info['tokens'],
            entity_info['label'],
            entity_info.get('start_ix', 0)  # Assuming start_ix is the key for the starting index
        )
        entities_list.append(entity)
    
    return entities_list


In [ ]:
true = prepare_target_from_radgrapgh(data)
pred = prepare_result_from_text(pred)

from nervaluate import Evaluator

evaluator = Evaluator(true, pred, tags=['LOC', 'PER'])

# Returns overall metrics and metrics for each tag
results, results_per_tag, result_indices, result_indices_by_tag = evaluator.evaluate()

print(results)

# ner_evaluation library
from ner_evaluation import compute_metrics


In [10]:
from ner_eval import compute_metrics
from ner_eval import Entity
import pprint   

true = [Entity(e_type='MISC', start_offset=12, end_offset=12),
        Entity(e_type='LOC', start_offset=15, end_offset=15),
        Entity(e_type='PER', start_offset=37, end_offset=39),
        Entity(e_type='ORG', start_offset=45, end_offset=46)]

pred = [Entity(e_type='MISC', start_offset=12, end_offset=12),
        Entity(e_type='LOC', start_offset=15, end_offset=15),
        Entity(e_type='PER', start_offset=37, end_offset=39),
        Entity(e_type='LOC', start_offset=45, end_offset=46)]

tags = ['LOC', 'PER', 'ORG', 'MISC']


results, evaluation_agg_entities_type = compute_metrics(true, pred, tags)

pprint.pprint(results["strict"])

{'actual': 4,
 'correct': 3,
 'incorrect': 1,
 'missed': 0,
 'partial': 0,
 'possible': 4,
 'precision': 0,
 'recall': 0,
 'spurious': 0}


## nervaluate

In [11]:
true = [
    [{"label": "PER", "start": 2, "end": 4}],
    [{"label": "LOC", "start": 1, "end": 2},
     {"label": "LOC", "start": 3, "end": 4}]
]

pred = [
    [{"label": "PER", "start": 2, "end": 4}],
    [{"label": "LOC", "start": 1, "end": 1},
     {"label": "LOC", "start": 3, "end": 4}]
]

from nervaluate import Evaluator

evaluator = Evaluator(true, pred, tags=['LOC', 'PER'])

# Returns overall metrics and metrics for each tag

results, results_per_tag, result_indices, result_indices_by_tag = evaluator.evaluate()

import pprint
pprint.pprint(results)

2024-10-21 10:41:51 root DEBUG: Imported 2 predictions for 2 true examples


{'ent_type': {'actual': 3,
              'correct': 3,
              'f1': 1.0,
              'incorrect': 0,
              'missed': 0,
              'partial': 0,
              'possible': 3,
              'precision': 1.0,
              'recall': 1.0,
              'spurious': 0},
 'exact': {'actual': 3,
           'correct': 2,
           'f1': 0.6666666666666666,
           'incorrect': 1,
           'missed': 0,
           'partial': 0,
           'possible': 3,
           'precision': 0.6666666666666666,
           'recall': 0.6666666666666666,
           'spurious': 0},
 'partial': {'actual': 3,
             'correct': 2,
             'f1': 0.8333333333333334,
             'incorrect': 0,
             'missed': 0,
             'partial': 1,
             'possible': 3,
             'precision': 0.8333333333333334,
             'recall': 0.8333333333333334,
             'spurious': 0},
 'strict': {'actual': 3,
            'correct': 2,
            'f1': 0.6666666666666666,
      

# NER with RE

In [ ]:
one_shot = {
            "1": {
                "tokens": "Lungs",
                "label": "ANAT-DP",
                "relations": []
            },
            "2": {
                "tokens": "clear",
                "label": "OBS-DP",
                "relations": [
                    [
                        "located_at",
                        "1"
                    ]
                ]
            },
            "3": {
                "tokens": "Normal",
                "label": "OBS-DP",
                "relations": [
                    [
                        "located_at",
                        "4"
                    ],
                    [
                        "located_at",
                        "5"
                    ],
                    [
                        "located_at",
                        "7"
                    ]
                ]
            },
            "4": {
                "tokens": "cardiomediastinal",
                "label": "ANAT-DP",
                "relations": []
            },
            "5": {
                "tokens": "hilar",
                "label": "ANAT-DP",
                "relations": []
            },
            "6": {
                "tokens": "silhouettes",
                "label": "ANAT-DP",
                "relations": [
                    [
                        "modify",
                        "4"
                    ],
                    [
                        "modify",
                        "5"
                    ]
                ]
            },
            "7": {
                "tokens": "pleural",
                "label": "ANAT-DP",
                "relations": []
            },
            "8": {
                "tokens": "surfaces",
                "label": "ANAT-DP",
                "relations": [
                    [
                        "modify",
                        "7"
                    ]
                ]
            }
    }